In [1]:
import pandas as pd

raw_data_1 = pd.read_json('./ppg-episode-transcripts-1.json', encoding='utf-8')
raw_data_2 = pd.read_json('./ppg-episode-transcripts-2.json', encoding='utf-8')

In [2]:
raw_data = pd.concat([raw_data_1, raw_data_2])

raw_data.head()

,episode,dialog
0,'Twas the Fight Before Christmas,"[Narrator: 'Twas the city of Townsville, and a..."
1,A Documentary,[Brian Larsen: The city of Townsville. This is...
2,A Made Up Story,[Narrator: The city of Townsville! Where some...
3,A Very Special Blossom,[Narrator: The city of Townsville. And it's a ...
4,Air Buttercup,[The girls are seen eating lunch in the cafete...


In [3]:
raw_data.describe()

,episode,dialog
count,155,155
unique,155,155
top,The Headsucker's Moxy,[Narrator: The city of Townsville! A secure pl...
freq,1,1


In [4]:
# Drop all
# - Animated shorts
# - Movies
# - PPG Z episodes
episodes_to_drop = [
   'a comedy of terrors', # powerpuff girls Z
    'enter the entourage',  # powerpuff girls Z
    "it's all because of him", # powerpuff girls Z
    'powerpuff girls to the rescue', # powerpuff girls Z

   'air buttercup', # Animated short, 2016
   "bubbles' beauty blog", # Animated short, 2016
   'ping pong z', # Animated short, 2016
   'run, blossom, run!', # Animated short, 2016
    "i'll be bake", # Animated short, 2017
    'mojo builds a shelf', # Animated short, 2017
    'the powerpuff girls rule!!!', # TV short, 2008

   'crime 101', # Pilot, 1995/1996
   'meat fuzzy lumkins', # Pilot, 1995

   'whoopass stew' # Movie, their first-ever appearance, 1992
   'the powerpuff girls movie', # Movie, 2002
   "'twas the fight before christmas",  # Movie, 2003
   'powerpuff girls special: dance pantsed', # Movie (weird style), 2014

   'ttg v ppg', # Teen titans, not ppg 
]

raw_data = raw_data[~raw_data['episode'].str.lower().isin(episodes_to_drop)]

In [5]:
all_lines = raw_data.explode(column='dialog')
all_lines = all_lines.reset_index(drop=True)
all_lines.head()

,episode,dialog
0,A Documentary,Brian Larsen: The city of Townsville. This is ...
1,A Documentary,"Brian: Oh, you might know them as those three ..."
2,A Documentary,"Brian: I'm Brian, and this is my documentary. ..."
3,A Documentary,Brian: Sugar... spice... and everything nice.
4,A Documentary,"Brian: Yep, those are pretty normal ingredient..."


In [6]:
all_lines[all_lines['dialog'].isna()]['episode']

11879    Small World: Heart to Heartstone
Name: episode, dtype: object

In [7]:
speakers = all_lines['dialog'].str.extract("^([.,*#-'/\w\s]*):")

all_lines['speaker'] = speakers
all_lines['speaker'] = all_lines['speaker'].str.strip()
all_lines['speaker'] = all_lines['speaker'].str.lower()
all_lines['speaker'] = all_lines['speaker'].str.strip()
speakers.nunique()

0    758
dtype: int64

In [8]:
all_lines['dialog'].head(50)

0     Brian Larsen: The city of Townsville. This is ...
1     Brian: Oh, you might know them as those three ...
2     Brian: I'm Brian, and this is my documentary. ...
3         Brian: Sugar... spice... and everything nice.
4     Brian: Yep, those are pretty normal ingredient...
5     Brian: The city of Townsville. A great city wi...
6     Brian: The newly renovated and revitalized Old...
7     Brian: Although this bus looks better than tha...
8     Brian: There are other reminders and countless...
9     Brian: Their photos have graced newspapers and...
10    Brian: They have merchandising... and even the...
11    Brian: My first visit was unannounced... and j...
12    Brian: the girls flew away. Three bright strea...
13    Brian: Okay... I'm driving up through the city...
14    Brian: I keep trying to get closer, but the ba...
15    Brian: I'm- I'm okay, now I'm gonna get out of...
16    Brian: And just when I thought I was going to ...
17    Brian: And although I played this cat- and

In [9]:
dialog_lines = all_lines['dialog']
dialog_lines = dialog_lines.reset_index(drop=True)
dialog_lines

0        Brian Larsen: The city of Townsville. This is ...
1        Brian: Oh, you might know them as those three ...
2        Brian: I'm Brian, and this is my documentary. ...
3            Brian: Sugar... spice... and everything nice.
4        Brian: Yep, those are pretty normal ingredient...
                               ...                        
16245                           Buttercup: Correct, Mayor.
16246                                          Mayor: Huh?
16247                               Bubbles: That’s right.
16248                                             Mayor: …
16249    Narrator: And so once again the day— My head h...
Name: dialog, Length: 16250, dtype: object

In [10]:
def process_dialog(dialog):
    if isinstance(dialog, str):
        parts = dialog.split(': ')
        if len(parts) > 1:
            return ' '.join(parts[1:])
        elif len(parts) == 1:
            return parts[0]
    else:
        return None
    
dialog_without_speaker = dialog_lines.apply(process_dialog)

In [11]:
dialog_without_speaker.head(50)

0     The city of Townsville. This is where the Powe...
1     Oh, you might know them as those three cute li...
2     I'm Brian, and this is my documentary. And you...
3                Sugar... spice... and everything nice.
4     Yep, those are pretty normal ingredients, all ...
5     The city of Townsville. A great city with many...
6     The newly renovated and revitalized Old Townsv...
7     Although this bus looks better than that corpo...
8     There are other reminders and countless monume...
9     Their photos have graced newspapers and tabloi...
10    They have merchandising... and even their own ...
11    My first visit was unannounced... and just as ...
12    the girls flew away. Three bright streaks cut ...
13    Okay... I'm driving up through the city, and t...
14    I keep trying to get closer, but the battle ke...
15    I'm- I'm okay, now I'm gonna get out of my car...
16    And just when I thought I was going to capture...
17    And although I played this cat- and- mouse

In [12]:
all_lines['dialog'] = dialog_without_speaker
all_lines['dialog']

0        The city of Townsville. This is where the Powe...
1        Oh, you might know them as those three cute li...
2        I'm Brian, and this is my documentary. And you...
3                   Sugar... spice... and everything nice.
4        Yep, those are pretty normal ingredients, all ...
                               ...                        
16245                                      Correct, Mayor.
16246                                                 Huh?
16247                                        That’s right.
16248                                                    …
16249            And so once again the day— My head hurts!
Name: dialog, Length: 16250, dtype: object

In [13]:
unique_speakers = all_lines['speaker'].unique()

for speaker in unique_speakers:
    if isinstance(speaker, str) and ',' in speaker:
        print(speaker)

bubbles, buttercup
blossom, buttercup
kids, blossom
slim, junior
mojo, princess, boys
professor, blossom, buttercup
brick, butch
blossom, bubbles
teen blossom, teen buttercup
girl 2, boy 1
blossom/ms. bellum, buttercup/professor
mayor, ms. bellum
mojo, 'him'
bossman, bossman 2
slim, slim 2
junior, juniors 2 and 3
driver, robbers
girls, duplicates
kids, parents
stan, sandra
fields, anthropist, blind
class, ms. keane
girls, professor
professor, ms. keane
ace, billy, arturo
fuzzy, princess, 'him'
kim, jeff, mary
joey, kim
necktie wearing monster, waitress
man 9, bass monster
gang, fuzzy, 'him'
bertha, beatrice
lloyd, floyd
ace, snake, lil arturo, big billy
002, 017
nephews, fuzzy
ace, snake, arturo, billy
ace, snake
boys 2, 3, 4
monty, slim
bubbles, buttercup and the professor
girls, robin
crowd, professor, ms. bellum
governor, yokel, lummox
lil arturo, snake
snake, arturo, billy


In [14]:
# Save one as second speaker
double_speakers = [    
    "blossom and buttercup",
    "blossom and bubbles",
    "the professor and blossom",
    "both blossom and buttercup",
    "buttercup and blossom",
    "blossom and the professor",
    "bubbles and blossom",
    "bubbles and buttercup",
    "buttercup and bubbles",
    "professor and buttercup",
    "professor utonium and javier xavier",
    "man and woman",
    "girls and erica",
    "momoko and miyako",
    "kaoru and miyako",
    "ms. bellumold woman 1",
    "ms. keaneold woman 2",
    "mayor and ms. bellum",
    "blossom/ms. bellum",
    "ms. bellum/blossom",
    "buttercup/green batman",
    "bubbles/bubbs wonder",
    "buttercup/professor",
    "professor/buttercup",
    "blossom/ms. bellum",
    "bubbles/mayor",
    "mayor/bubbles",
    "ms. bellum/blossom",
    "girl 1/ace",
    "girl 2/grubber",
    "girl 3/snake",
    "old woman/mojo",
    "mojo/old woman",
    "blossom/dachshund",
    "bubbles/centipede",
    "buttercup/cobra",
    "professor/older girl",
    "mojo/fish",
    "professor/ugly baby",
    "professor/nurse",
    "professor/astronaut",
    "professor/pink creature",
    "professor/morbid woman",
    "bubbles/narrator",
    "narrator/bubbles",
    "blossom/bubbles",
    "octi/him",
    "buttercup/bubblecup",
    "bubbles/children",
    "old man/janitor",
    "eyes/blonde hair covered boy",
    "blossom/liberty belle",
    "bubbles/harmony bunny",
    "buttercup/mange",
    "blossom/buttercup",
    "bubbles, buttercup",
"girls, professor",
"blossom, buttercup",
"blossom, bubbles and buttercup",
"kids, blossom",
"slim, junior",
"mojo, princess, boys",
"blossom, bubbles, buttercup and the professor",
"blossom, bubbles, and buttercup",
"professor, blossom, buttercup",
"brick, butch",
"blossom, bubbles",
"teen blossom, teen buttercup",
"girl 2, boy 1",
"skinny, tiny",
"blossom/ms. bellum, buttercup/professor",
"mayor, ms. bellum",
"mojo, 'him'",
"bossman, bossman 2",
"slim, slim 2",
"junior, juniors 2 and 3",
"bubbles, buttercup, random boy, random girl, another random boy, robin snyder, another random boy, two random boys, blossom, jared, and barry",
"random girl , random boy , random boy , buttercup  on the left, jared , and robin , blossom , and two boys",
"blossom, buttercup, jared, barry, and robin",
"chad, man with glasses, and pink haired lady",
"driver, robbers",
"girls, duplicates",
"kids, parents",
"stan, sandra",
"fields, anthropist, blind",
"class, ms. keane",
"pug faced paulie, scared",
"momoko, miyako, and kaoru",
"professor, ms. keane",
"ace, billy, arturo",
"fuzzy, princess, 'him'",
"kim, jeff, mary",
"joey, kim",
"man 9, bass monster",
"gang, fuzzy, 'him'",
"bertha, beatrice",
"lloyd, floyd",
"ace, snake, lil' arturo, big billy",
"002, 017",
"ace, snake, arturo, billy",
"ace, snake",
"boys 2, 3, 4",
"monty, slim",
"bubbles, buttercup and the professor",
"girls, robin",
"barry, bubbles, and buttercup",
"bubbles, blossom",
"many monkeys, chimpanzees, gorillas, orangutans, and even an uakari",
"crowd, professor, ms. bellum",
"governor, yokel, lummox",
"octi/'him'",
"'him'/ms. keane",
"blossom & buttercup",
"blossom & bubbles",
"bubbles & blossom",
"blossom & professor utonium",
"bubbles & buttercup",
"buttercup & bubbles",
"spikey haired boy & eyes/blonde hair covered boy",
"donny & chelsea"
]

blossom_values = [ 
    "blossom #2",
    "blosoom",
    "all blossoms",
    "blossom  and blossom",
    "the blossoms",
    "siren blossom",
    "teen blossom",
    "blossom as she fights the bunnies",
    "knockoff blossom",
    "survivor reject blossom",
    "angel blossom",
    "robot blossom",
    "robo blossom",
    "lice blossom",
    "french renaissance blossom",
    "old blossom",
    "blossom, in the shop",
    "blossom, to bubbles",
    "blossom's thoughts",
    "blossom ",
    "both blossom"
]

bubbles_values = [
    "bubbles",
    "bubbes",
    "dark bubbles",
    "teen bubbles",
    "knockoff bubbles",
    "robo bubbles",
    "french renaissance bubbles",
    "bubbles and doubles",
    "old bubbles",
    "bubblecup",
    "bubbles' voice",
    "bubbles #2"
]

buttercup_values = [
    "buttercup double",
    "buttecup",
    "real buttercup",
    "buttercup on video",
    "teen buttercup",
    "knockoff buttercup",
    "reject buttercup",
    "angel buttercup",
    "robo buttercup",
    "flashback buttercup",
    "old buttercup",
    "buttercup, to herself",
    "buttercup, disguised",
    "buttercup, as she accidentaly reveals herself",
    "buttercup, scared",
    "and buttercup",
    "buttercup  on the left",
    "buttercup's thoughts",
    "buttercup #2"
]

mojo_values = [
    "mojo",
    "jojo",
    "'mojo'",
    "'mojo' ",
    "mojo'",
    "mojo's thoughts"
]

professor_values = [
    "professor",
    "dream professor",
    "memory professor",
    "professor utonium",
    "kid professor",
    "young prof",
    "student prof",
    "the professor",
    "flashback professor",
]

mayor_values = [
    "mayor",
    "the mayor",
    "younger mayor",
    "mayor mayer",
    "mayor of new townsville",
    "lice mayor",
    "citiesville mayor",
    "dream mayor"
]

him_values = [
    "him",
    "'him'",
    "memory him",
    "black spored him"
]

narrator_values = [
    "narrator",
    "male narrator",
    "man voiceover",
]

princess_values = [
    "princess morebucks",
    "princess",
    "princess morbucks",
    "morbucks",
    "morbucks' heart"
]

fuzzy_values = [
    "fuzzy",
    "fuzzy lumpkins"
]

brick_values = [
    "brick",
    "teen brick"
]

mike_values = [
    "brickowski",
    "mike"
]

woman_values = [
    "old woman",
    "woman",
    "muscle woman",
    "woman in the suit",
    "policewoman",
    "woman 2",
    "woman 3",
    "brown skinned woman",
    "woman 1",
    "young woman",
    "adult teenage woman",
    "woman 4",
    "woman 5",
    "woman 6",
    "woman 7",
    "iss woman",
    "cowgirl woman",
    "backgrounder woman",
    "fat woman",
    "fainting woman",
    "nature woman",
    "jewel owner woman",
    "jamaican woman",
    "curler woman",
    "woman 8",
    "woman 9",
    "repairwoman",
    "morbid woman",
    "woman #2",
    "woman #3",
    "woman #1",
    "woman's voice",
    "lunch lady",
    "and pink haired lady",
    "rocket a.i. lady",
    "cleaning lady",
    "video lady",
    "lady" #1,
    "old lady",
    "banklady",
    "news lady",
    "elderly lady",
    "anchor lady",
    "lady",
    "baby lady",
    "random lady",
    "lady #1"
]

man_values = [
    "man",
    "policeman",
    "old man",
    "swan boat man",
    "barber man",
    "man 1",
    "man 2",
    "man 3",
    "man in court",
    "man in a blue long sleeved robe",
    "old man 1",
    "old man 2",
    "old man 3",
    "man with glasses",
    "policeman 1",
    "policeman 2",
    "security man 1",
    "security man 2",
    "postman",
    "renaissance man",
    "delivery man",
    "man with a green hat",
    "man wit a green hat",
    "man in the space suit",
    "running man",
    "pointing man",
    "manager",
    "french man with a french hat",
    "brown haired man with glasses",
    "man 4",
    "man 5",
    "hotdog man",
    "man 6",
    "man 7",
    "man 8",
    "man 9",
    "man 10",
    "man 11",
    "abs man",
    "commercial man",
    "prison man",
    "salesman",
    "henchman",
    "mailman",
    "backgrounder man",
    "bank manager",
    "pizza man",
    "rainbow man",
    "bagman",
    "gunman",
    "short man",
    "newsman",
    "jewel owner man",
    "smooth man",
    "anchorman",
    "businessman",
    "french renaissance man",
    "fireman 1",
    "man 12",
    "man 13",
    "man 14",
    "fireman 2",
    "man being stung by bees",
    "foam finger man",
    "man in overalls",
    "man in trough",
    "male actor",
    "male chorus",
    "male voice",
    "male staff member",
    "man #1",
    "man #2",
    "man #3",
    "man #4",
]

bliss_values = [
    "memory bliss",
    "bliss",
    "memory teenage bliss",
    "young memory bliss"
]

monster_values = [
    "monster",
    "pink tentacled monster",
    "green monster",
    "bass monster",
    "female fish monster",
    "red lizard monster",
    "lawyer monster",
    "monsters",
    "fancy monster",
    "purple octopus",
    "dragon",
    "pincered monster",
    "necktie wearing monster",
    "oneeyed monster",
    "singledout monster",
    "monster #1",
    "crowd of monsters"
]

boy_values = [
    "boy",
    "huge boy",
    "pimple faced boy",
    "boy 1",
    "boy 2",
    "boy 3",
    "boy 4",
    "nerd boy",
    "bigger boy",
    "smaller boy",
    "boy with brown spiky hair",
    "man boy",
    "manboy",
    "spikey haired boy",
    "freckle faced boy",
    "boy 5",
    "boy 6",
    "boy 7",
    "boy nerd",
    "green haired boy with rapper glasses",
    "french renaissance orange haired boy",
    "redhead boy",
    "'injured' boy",
    "brownskinned boy",
    "random boy",
    "another random boy",
    "two random boys",
    "random boy ",
    "and two boys",
    "spikey haired boy & eyes/blonde hair covered boy",
    "blonde hair covered boy",
    "boys 2",
    "boy #1",
]

ppg_values = [
    "all 3 girls",
    "all three girls",
    "girls",
    "the girls",
    "old girls",
    "the 3 girls",
    "all girls",
    "powerpuff girls z",
    "the 4 girls"
]

rrb_values = [
    "boys"
]

girl_values = [
    "girl",
    "cool girl",
    "girl with braces",
    "girl 1",
    "girl 2",
    "girl 3",
    "blonde teenage girl",
    "blonde girl",
    "girl with glasses",
    "girl without glasses",
    "2 girls",
    "cool girl 1",
    "cool girl 3",
    "cool girl 2",
    "all 3 cool girls",
    "girl with brown hair",
    "brown haired girl",
    "cowgirl",
    "friendly girl",
    "derbytante girls",
    "spirit girl",
    "mail girl",
    "flower girl",
    "girl student",
    "girl 4",
    "2 unnamed girls",
    "unnamed girl with a pink shirt",
    "teenage girl 1",
    "teenage girl 3",
    "teenage girls",
    "pimple girl",
    "older girl",
    "random girl",
    "random girl ",
    "girl #1",
    "girl #2",
    "girl #3",
]

miyako_values = [
    "memory miyako",
    "miyako",
    "flashback miyako",
]

sapna_values = [
    "sapna,"
    "dream sapna",
    "spider sapna"
]

kaoru_values = [
    "kaoru matsubara"
]

announcer_values = [
    "space towtruck and the infinite jumper cables movie announcer",
    "announcer",
    "announcer 1",
    "announcer 2",
    "another announcer",
    "radio announcer",
    "tv announcer",
    "space towtruck announcer",
    "male pageant announcer",
    "video game announcer",
]

keane_values = [
    "ms. keane",
    "young keane",
    "keanes",
    "keane 2",
    "keane 1",
    "miss keane",
    "french renaissance ms. keane",
    "ms. keane"
]

bellum_values = [
    "ms. bellum",
    "young bellum",
    "miss bellum",
    "dream ms. bellum",
    "ms.bellum",
    "ms. bellum"
]

billy_values = [
    "billy",
    "big billy",
    " billy",
    " big billy",
]

arturo_values = [
    "lil arturo",
    " lil arturo",
    " arturo"
]

butch_values = [
    "butch",
    "teen butch"
]

boomer_values = [
    "boomer",
    "teen boomer"
]

talking_dog_values = [
    "talking dog",
    "talking  dog",
    "talking dog billboard"
]

junior_values = [
    "junior",
    "junior 4"
]

bossman_values = [
    "bossman",
    "bossman 3",
    "bossman 2"
]

dick_values = [
    "dick",
    "student dick"
]

marianne_values = [
    'mrs. smith'
]

harold_values = [
    'mr. smith'
]

sedusa_values = [
    'ima'
]

reporter_values = [
    "reporter",
    "reporter 2",
    "female reporter",
    "male reporter",
    "radio reporter",
    "tv reporter 1",
    "tv reporter 2",
    "tv reporter 3",
    "various reporters",
    "tv reporter 4",
]

thief_values = [
    "robber",
    "robbers",
    "'robber' 1",
    "'robber' 2",
    "thin robber",
    "fat robber",
    "short robber",
    "robber 1",
    "robber 2",
    "thief"
]

kid_values = [
    "kids",
    "kid",
    "beefy kid",
    "fat kid",
    "kid 1",
    "kid 2",
    "kid 3",
    "wedgie kid",
]

snake_values = [
    " snake"
]

cop_values = [
    "chief of police",
    "blond 'policeman'",
    "policeman #1",
    "policeman #2",
    "cop",
    "fat cop",
    "thin cop",
    "radio cop",
    "cop 2",
    "cop 1",
]

In [15]:
import re

# All double speakers in consistent format (comma-separated)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: re.sub(r'(/|\sand\s|,\s|\s&\s)', ',', x) if x in double_speakers else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: re.sub('/', ',', x) if x in double_speakers else x)

all_lines.loc[all_lines['speaker'] == 'ms. bellumold woman 1', 'speaker'] = 'ms. bellum,woman'
all_lines.loc[all_lines['speaker'] == 'ms. keaneold woman 2', 'speaker'] = 'ms. keane,woman'

double_speaker_items = all_lines[all_lines['speaker'].isin(double_speakers)]
double_speaker_items['speaker']

Series([], Name: speaker, dtype: object)

In [16]:
len(all_lines['speaker'])

16250

In [17]:
new_double_speakers = all_lines[all_lines['speaker'].str.contains(',', na=False)]
len(new_double_speakers['speaker'].value_counts())

81

In [18]:
all_lines['speaker'] = all_lines['speaker'].str.split(',')
all_lines = all_lines.explode('speaker')
all_lines = all_lines.reset_index(drop=True)
all_lines

,episode,dialog,speaker
0,A Documentary,The city of Townsville. This is where the Powe...,brian larsen
1,A Documentary,"Oh, you might know them as those three cute li...",brian
2,A Documentary,"I'm Brian, and this is my documentary. And you...",brian
3,A Documentary,Sugar... spice... and everything nice.,brian
4,A Documentary,"Yep, those are pretty normal ingredients, all ...",brian
...,...,...,...
16656,The Headsucker's Moxy,"Correct, Mayor.",buttercup
16657,The Headsucker's Moxy,Huh?,mayor
16658,The Headsucker's Moxy,That’s right.,bubbles
16659,The Headsucker's Moxy,…,mayor


In [19]:
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'blossom' if x in blossom_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'bubbles' if x in bubbles_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'buttercup' if x in buttercup_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'professor' if x in professor_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'mayor' if x in mayor_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'mojo jojo' if x in mojo_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'narrator' if x in narrator_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'him' if x in him_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'princess morbucks' if x in princess_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'fuzzy' if x in fuzzy_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'woman' if x in woman_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'man' if x in man_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'brick' if x in brick_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'bliss' if x in bliss_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'monster' if x in monster_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'boy' if x in boy_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'girl' if x in girl_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'powerpuff girls' if x in ppg_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'rowdyruff boys' if x in rrb_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'sapna' if x in sapna_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'miyako' if x in miyako_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'kaoru' if x in kaoru_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'announcer' if x in announcer_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'ms keane' if x in keane_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'ms bellum' if x in bellum_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'billy' if x in billy_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'arturo' if x in arturo_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'butch' if x in butch_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'boomer' if x in boomer_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'talking dog' if x in talking_dog_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'junior' if x in junior_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'bossman' if x in bossman_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'marianne' if x in marianne_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'harold' if x in harold_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'sedusa' if x in sedusa_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'reporter' if x in reporter_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'thief' if x in thief_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'kid' if x in kid_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'snake' if x in snake_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'cop' if x in cop_values else x)
all_lines['speaker'] = all_lines['speaker'].apply(lambda x: 'mike' if x in mike_values else x)

In [20]:
unique_speakers = all_lines['speaker'].unique()

for speaker in unique_speakers:
    if isinstance(speaker, str) and "dog" in speaker:
        print(speaker)

talking dog
dog #2
cellmate dog
dog puppet
dog


In [21]:
all_lines['speaker'].value_counts().head(60)

blossom              2927
buttercup            2058
bubbles              1948
professor            1475
narrator             1077
mojo jojo             869
mayor                 753
powerpuff girls       449
ms keane              329
him                   249
princess morbucks     235
man                   219
ms bellum             181
ace                   169
woman                 152
brick                 142
fuzzy                 111
bossman                91
billy                  82
sedusa                 77
talking dog            69
dick                   62
monster                62
bernie                 60
announcer              54
marianne               53
thief                  52
butch                  50
boomer                 50
girl                   46
brian                  46
mike                   46
robin                  46
boy                    45
major glory            45
arturo                 42
mitch                  42
snake                  41
whitfield   

In [22]:
# Identify episodes with only 1 line, probably we need to filter them out
all_lines['episode'].value_counts()

Criss Cross Crisis                  281
Superfriends                        240
Power-Noia                          227
Members Only                        225
Film Flam                           222
                                   ... 
What's the Big Idea?                 59
Neighbor Hood                        55
Down 'n Dirty                        53
Small World: Abra-Disaster           31
Small World: Heart to Heartstone      1
Name: episode, Length: 142, dtype: int64

In [23]:
import numpy as np

# Create a villain column
conditions = [
    all_lines['speaker'] == 'mojo jojo',
    all_lines['speaker'] == 'him',
    all_lines['speaker'] == 'sedusa',
    all_lines['speaker'] == 'princess morbucks',
    all_lines['speaker'] == 'dick',
    all_lines['speaker'] == 'bernie',
    all_lines['speaker'] == 'sandman',
    all_lines['speaker'] == 'lenny',
    all_lines['speaker'] == 'femme fatale',
    all_lines['speaker'] == 'gnome',
    all_lines['speaker'] == 'fuzzy',
    all_lines['speaker'].isin(['brick', 'boomer', 'butch', 'rowdyruff boys']), 
    all_lines['speaker'].isin(['ace', 'grubber', 'billy', 'snake', 'arturo']),
    all_lines['speaker'].isin(['bossman', 'junior', 'slim', 'amoeba boys']),
    all_lines['speaker'].isin(['marianne', 'harold', 'julie', 'bud']),
]
values = [
    'mojo jojo', 
    'him',
    'sedusa',
    'princess morbucks',
    'dick',
    'bernie',
    'sandman',
    'lenny',
    'femme fatale',
    'gnome',
    'fuzzy',
    'rowdyruff boys',
    'gangreen gang',
    'amoeba boys',
    'smith family'
]

# Use numpy's where function to create the new column
all_lines['villain'] = np.select(conditions, values, default=None)
all_lines['villain'].head()

0    None
1    None
2    None
3    None
4    None
Name: villain, dtype: object

In [24]:
all_lines['villain'].value_counts()

mojo jojo            869
gangreen gang        350
rowdyruff boys       261
him                  249
princess morbucks    235
amoeba boys          157
fuzzy                111
smith family         105
sedusa                77
dick                  62
bernie                60
lenny                 25
femme fatale          23
sandman               14
gnome                  7
Name: villain, dtype: int64

In [25]:
seasons_1998_raw = pd.read_json('./ppg-seasons-1998.json', encoding='utf-8')
seasons_2016_raw = pd.read_json('./ppg-seasons-2016.json', encoding='utf-8')

seasons_2016_raw.head()

,year,season,episode_nr,title,description
0,2016,1,1,Escape from Monster Island,While Bubbles tries to decide whether to take ...
1,2016,1,2,Princess Buttercup,"Buttercup starts hanging out with a bad crowd,..."
2,2016,1,3,The Stayover,"Recovering from a candy hangover, Blossom and ..."
3,2016,1,4,Painbow,A rainbow makes everyone in Townsville unnatur...
4,2016,1,5,"Horn, Sweet Horn","At Bubbles' request, the Professor tries to ma..."


In [26]:
seasons_raw = pd.concat([seasons_1998_raw, seasons_2016_raw])
seasons_raw.head(10)

,year,season,episode_nr,title,description
0,1998,1,0,A Sticky Situation!,Begins with the same opening sequence as the s...
1,1998,1,1,"Monkey See, Doggie Do/Mommy Fearest","In ""Monkey See, Doggie Do"", Mojo Jojo swipes s..."
2,1998,1,2,Insect Inside/Powerpuff Bluff,"""Insect Inside"" The city of Townsville becomes..."
3,1998,1,3,Octi Evil/Geshundfight,"""Octi-Evil"" Him is out to break up the Powerpu..."
4,1998,1,4,Buttercrush/Fuzzy Logic,"""Buttercrush"" Buttercup develops a crush on Ac..."
5,1998,1,5,Boogie Frights/Abracadaver,"In ""Boogie Frights"", the Boogie Man comes up w..."
6,1998,1,6,Telephonies/Tough Love,"""Telephonies"" The Gangreen Gang gets hold of t..."
7,1998,1,7,Major Competition/Mr. Mojo's Rising,"""Major Competition"" The Powerpuff Girl's place..."
8,1998,1,8,Paste Makes Waste/Ice Sore,"In ""Paste Makes Waste"", one of the Pokey Oaks ..."
9,1998,1,9,Bubblevicious/The Bare Facts,Bubbles is tired of being the cute and wants t...


In [27]:
seasons_raw['title'].head(50)

0                                   A Sticky Situation!
1                   Monkey See, Doggie Do/Mommy Fearest
2                         Insect Inside/Powerpuff Bluff
3                                Octi Evil/Geshundfight
4                               Buttercrush/Fuzzy Logic
5                            Boogie Frights/Abracadaver
6                                Telephonies/Tough Love
7                   Major Competition/Mr. Mojo's Rising
8                            Paste Makes Waste/Ice Sore
9                          Bubblevicious/The Bare Facts
10                              Cat Man Do/Impeach Fuzz
11            Just Another Manic Mojo/Mime for a Change
12                                   The Rowdyruff Boys
13                                         Uh Oh Dynamo
14            Stuck Up, Up, and Away/Schoolhouse Rocked
15                           Collect Her/Supper Villain
16                     Birthday Bash/Too Pooped to Puff
17                       Beat Your Greens/Down '

In [28]:
seasons = seasons_raw.copy()

seasons['title'] = seasons['title'].str.lower()
seasons['title'] = seasons['title'].str.split('/')
seasons['title_1'] = seasons['title'].str[0]
seasons['title_2'] = seasons['title'].str[1]
all_lines['episode'] = all_lines['episode'].str.lower()
seasons.head(10)

,year,season,episode_nr,title,description,title_1,title_2
0,1998,1,0,[a sticky situation!],Begins with the same opening sequence as the s...,a sticky situation!,NaN
1,1998,1,1,"[monkey see, doggie do, mommy fearest]","In ""Monkey See, Doggie Do"", Mojo Jojo swipes s...","monkey see, doggie do",mommy fearest
2,1998,1,2,"[insect inside, powerpuff bluff]","""Insect Inside"" The city of Townsville becomes...",insect inside,powerpuff bluff
3,1998,1,3,"[octi evil, geshundfight]","""Octi-Evil"" Him is out to break up the Powerpu...",octi evil,geshundfight
4,1998,1,4,"[buttercrush, fuzzy logic]","""Buttercrush"" Buttercup develops a crush on Ac...",buttercrush,fuzzy logic
5,1998,1,5,"[boogie frights, abracadaver]","In ""Boogie Frights"", the Boogie Man comes up w...",boogie frights,abracadaver
6,1998,1,6,"[telephonies, tough love]","""Telephonies"" The Gangreen Gang gets hold of t...",telephonies,tough love
7,1998,1,7,"[major competition, mr. mojo's rising]","""Major Competition"" The Powerpuff Girl's place...",major competition,mr. mojo's rising
8,1998,1,8,"[paste makes waste, ice sore]","In ""Paste Makes Waste"", one of the Pokey Oaks ...",paste makes waste,ice sore
9,1998,1,9,"[bubblevicious, the bare facts]",Bubbles is tired of being the cute and wants t...,bubblevicious,the bare facts


In [29]:
lines_with_seasons = pd.merge(all_lines, seasons, how='left', left_on='episode', right_on='title_1')
lines_with_seasons = pd.merge(lines_with_seasons, seasons, how='left', left_on='episode', right_on='title_2')

In [30]:
lines_with_seasons.head()

,episode,dialog,speaker,villain,year_x,season_x,episode_nr_x,title_x,description_x,title_1_x,title_2_x,year_y,season_y,episode_nr_y,title_y,description_y,title_1_y,title_2_y
0,a documentary,The city of Townsville. This is where the Powe...,brian larsen,None,1998.0,5.0,9.0,"[a documentary, girls gone mild]",".""Documentary"" A film student attempts to do a...",a documentary,girls gone mild,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a documentary,"Oh, you might know them as those three cute li...",brian,None,1998.0,5.0,9.0,"[a documentary, girls gone mild]",".""Documentary"" A film student attempts to do a...",a documentary,girls gone mild,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a documentary,"I'm Brian, and this is my documentary. And you...",brian,None,1998.0,5.0,9.0,"[a documentary, girls gone mild]",".""Documentary"" A film student attempts to do a...",a documentary,girls gone mild,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,a documentary,Sugar... spice... and everything nice.,brian,None,1998.0,5.0,9.0,"[a documentary, girls gone mild]",".""Documentary"" A film student attempts to do a...",a documentary,girls gone mild,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a documentary,"Yep, those are pretty normal ingredients, all ...",brian,None,1998.0,5.0,9.0,"[a documentary, girls gone mild]",".""Documentary"" A film student attempts to do a...",a documentary,girls gone mild,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
lines_with_seasons.groupby(['season_y'])['episode'].nunique()

season_y
1.0    11
2.0    13
3.0    11
4.0     1
5.0    10
6.0    12
Name: episode, dtype: int64

In [32]:
lines_with_seasons.groupby(['season_x'])['episode'].nunique()

season_x
1.0    13
2.0    14
3.0    17
4.0    12
5.0    12
6.0    15
Name: episode, dtype: int64

In [33]:
lines_with_seasons.columns

Index(['episode', 'dialog', 'speaker', 'villain', 'year_x', 'season_x',
       'episode_nr_x', 'title_x', 'description_x', 'title_1_x', 'title_2_x',
       'year_y', 'season_y', 'episode_nr_y', 'title_y', 'description_y',
       'title_1_y', 'title_2_y'],
      dtype='object')

In [34]:
lines_with_seasons['season'] = np.where(lines_with_seasons['season_x'].notna(), lines_with_seasons['season_x'], lines_with_seasons['season_y'])
lines_with_seasons['episode_nr'] = np.where(lines_with_seasons['episode_nr_x'].notna(), lines_with_seasons['episode_nr_x'], lines_with_seasons['episode_nr_y'])
lines_with_seasons['description'] = np.where(lines_with_seasons['description_x'].notna(), lines_with_seasons['description_x'], lines_with_seasons['description_y'])
lines_with_seasons['title_1'] = np.where(lines_with_seasons['title_1_x'].notna(), lines_with_seasons['title_1_x'], lines_with_seasons['title_1_y'])
lines_with_seasons['title_2'] = np.where(lines_with_seasons['title_2_x'].notna(), lines_with_seasons['title_2_x'], lines_with_seasons['title_2_y'])
lines_with_seasons['year'] = np.where(lines_with_seasons['year_x'].notna(), lines_with_seasons['year_x'], lines_with_seasons['year_y'])
lines_with_seasons['season'].value_counts()

2.0    3006
6.0    2954
3.0    2922
1.0    2724
5.0    2584
4.0    2471
Name: season, dtype: int64

In [35]:
lines_with_seasons['season'].isna().sum()

0

In [36]:
lines_with_seasons.drop(columns=[
    'season_x', 'episode_nr_x',
       'title_x', 'description_x', 'title_1_x', 'title_2_x', 'season_y',
       'episode_nr_y', 'title_y', 'description_y', 'title_1_y', 'title_2_y', 'year_x', 'year_y' 
], inplace=True)

In [37]:
lines_with_seasons['episode_nr'] = np.where(lines_with_seasons['title_2'] == lines_with_seasons['episode'], lines_with_seasons['episode_nr'] + 0.2, lines_with_seasons['episode_nr'])
lines_with_seasons['episode_nr'] = np.where((lines_with_seasons['title_1'] == lines_with_seasons['episode']) & (lines_with_seasons['title_2'].notna()), lines_with_seasons['episode_nr'] + 0.1, lines_with_seasons['episode_nr'])

In [38]:
lines_with_seasons[lines_with_seasons['season'].isna()]['episode'].unique()

array([], dtype=object)

In [39]:
# NB They are all 2016, so for now I'm not too worried
lines_with_seasons[lines_with_seasons['dialog'].isna()]

,episode,dialog,speaker,villain,season,episode_nr,description,title_1,title_2,year
12171,small world: heart to heartstone,None,NaN,None,3.0,16.0,The Powerpuff Girls are sent to the arctic on ...,small world: heart to heartstone,NaN,2016.0


In [40]:
lines_with_seasons.dropna(subset=['dialog'], inplace=True)

In [41]:
lines_with_seasons[lines_with_seasons['dialog'].isna()]

,episode,dialog,speaker,villain,season,episode_nr,description,title_1,title_2,year


In [42]:
lines_with_seasons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16660 entries, 0 to 16660
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   episode      16660 non-null  object 
 1   dialog       16660 non-null  object 
 2   speaker      16524 non-null  object 
 3   villain      2605 non-null   object 
 4   season       16660 non-null  float64
 5   episode_nr   16660 non-null  float64
 6   description  16660 non-null  object 
 7   title_1      16660 non-null  object 
 8   title_2      12707 non-null  object 
 9   year         16660 non-null  float64
dtypes: float64(3), object(7)
memory usage: 1.4+ MB


In [43]:
lines_with_seasons.head()

,episode,dialog,speaker,villain,season,episode_nr,description,title_1,title_2,year
0,a documentary,The city of Townsville. This is where the Powe...,brian larsen,None,5.0,9.1,".""Documentary"" A film student attempts to do a...",a documentary,girls gone mild,1998.0
1,a documentary,"Oh, you might know them as those three cute li...",brian,None,5.0,9.1,".""Documentary"" A film student attempts to do a...",a documentary,girls gone mild,1998.0
2,a documentary,"I'm Brian, and this is my documentary. And you...",brian,None,5.0,9.1,".""Documentary"" A film student attempts to do a...",a documentary,girls gone mild,1998.0
3,a documentary,Sugar... spice... and everything nice.,brian,None,5.0,9.1,".""Documentary"" A film student attempts to do a...",a documentary,girls gone mild,1998.0
4,a documentary,"Yep, those are pretty normal ingredients, all ...",brian,None,5.0,9.1,".""Documentary"" A film student attempts to do a...",a documentary,girls gone mild,1998.0


In [44]:
# Fill all lines without a speaker with the speaker before. CHECK!!
no_speakers = lines_with_seasons[lines_with_seasons['speaker'].isna()]
no_speakers.head()

,episode,dialog,speaker,villain,season,episode_nr,description,title_1,title_2,year
2579,cop out,"Back to you there, Kevin.",NaN,None,3.0,5.2,"""Gettin' Twiggy With It"" Mitch is told to take...",gettin' twiggy with it,cop out,1998.0
3757,dream scheme,It is the setting of this little story.,NaN,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0
3759,dream scheme,Where people are dressed in PJ and robe.,NaN,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0
3760,dream scheme,"They kick off their slippers and climb into beds,",NaN,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0
3761,dream scheme,Fluff up their pillows and lay down their heads.,NaN,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0


In [45]:
no_speakers.count()

episode        136
dialog         136
speaker          0
villain          0
season         136
episode_nr     136
description    136
title_1        136
title_2        128
year           136
dtype: int64

In [46]:
lines_with_seasons['speaker'].fillna(method='ffill', inplace=True)

In [47]:
lines_with_seasons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16660 entries, 0 to 16660
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   episode      16660 non-null  object 
 1   dialog       16660 non-null  object 
 2   speaker      16660 non-null  object 
 3   villain      2605 non-null   object 
 4   season       16660 non-null  float64
 5   episode_nr   16660 non-null  float64
 6   description  16660 non-null  object 
 7   title_1      16660 non-null  object 
 8   title_2      12707 non-null  object 
 9   year         16660 non-null  float64
dtypes: float64(3), object(7)
memory usage: 1.4+ MB


In [48]:
previous_no_speakers = lines_with_seasons[lines_with_seasons.index.isin(no_speakers.index)]
previous_no_speakers.count()

episode        136
dialog         136
speaker        136
villain          0
season         136
episode_nr     136
description    136
title_1        136
title_2        128
year           136
dtype: int64

In [49]:
previous_no_speakers.iloc[:10, :]

,episode,dialog,speaker,villain,season,episode_nr,description,title_1,title_2,year
2579,cop out,"Back to you there, Kevin.",reporter,None,3.0,5.2,"""Gettin' Twiggy With It"" Mitch is told to take...",gettin' twiggy with it,cop out,1998.0
3757,dream scheme,It is the setting of this little story.,narrator,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0
3759,dream scheme,Where people are dressed in PJ and robe.,narrator,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0
3760,dream scheme,"They kick off their slippers and climb into beds,",narrator,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0
3761,dream scheme,Fluff up their pillows and lay down their heads.,narrator,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0
3763,dream scheme,Enter the Sandman to do them up right.,narrator,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0
3765,dream scheme,He pulls out a magical brown grain of sand.,narrator,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0
3767,dream scheme,"And out goes the victim, asleep for the night.",narrator,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0
3769,dream scheme,"The Sandman does his job, singing his ditty.",narrator,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0
3771,dream scheme,Putting you to sleep is what I'm about.,sandman,None,2.0,5.1,"""Dream Scheme"" The Sandman would like a little...",dream scheme,"you snooze, you lose",1998.0


In [50]:
lines_with_seasons['word_count_for_line'] = lines_with_seasons['dialog'].str.split(' ')
lines_with_seasons['word_count_for_line'] = lines_with_seasons['word_count_for_line'].apply(len)
lines_with_seasons[['dialog', 'word_count_for_line']].head(15)

,dialog,word_count_for_line
0,The city of Townsville. This is where the Powe...,11
1,"Oh, you might know them as those three cute li...",121
2,"I'm Brian, and this is my documentary. And you...",31
3,Sugar... spice... and everything nice.,5
4,"Yep, those are pretty normal ingredients, all ...",34
5,The city of Townsville. A great city with many...,31
6,The newly renovated and revitalized Old Townsv...,24
7,Although this bus looks better than that corpo...,22
8,There are other reminders and countless monume...,15
9,Their photos have graced newspapers and tabloi...,9


In [51]:
lines_1998 = lines_with_seasons[lines_with_seasons['year'] == 1998]

summed_speakers_1998 = lines_1998.groupby('speaker')['word_count_for_line'].sum().reset_index()
summed_speakers_1998 = summed_speakers_1998.sort_values('word_count_for_line', ascending=False).reset_index(drop=True)

# Add instances in which PPG speak all together to single girls, drop PPG column
pppg_value = summed_speakers_1998[summed_speakers_1998['speaker'] == 'powerpuff girls']['word_count_for_line'].values[0]
# summed_speakers_1998.loc[summed_speakers_1998['speaker'] == 'blossom', 'word_count_for_line'] += pppg_value
# summed_speakers_1998.loc[summed_speakers_1998['speaker'] == 'bubbles', 'word_count_for_line'] += pppg_value
# summed_speakers_1998.loc[summed_speakers_1998['speaker'] == 'buttercup', 'word_count_for_line'] += pppg_value
# summed_speakers_1998 = summed_speakers_1998[summed_speakers_1998['speaker'] != 'powerpuff girls']

summed_speakers_1998.head(50)

,speaker,word_count_for_line
0,blossom,21050
1,professor,14823
2,narrator,12839
3,buttercup,12609
4,bubbles,11962
5,mojo jojo,10387
6,mayor,7573
7,ms keane,2996
8,princess morbucks,2861
9,him,2684


In [52]:
summed_speakers_1998['word_count_for_line'].sum()

137275

In [53]:
lines_2016 = lines_with_seasons[lines_with_seasons['year'] == 2016]

summed_speakers_2016 = lines_2016.groupby('speaker')['word_count_for_line'].sum().reset_index()
summed_speakers_2016.sort_values('word_count_for_line', ascending=False).head(50).reset_index(drop=True)

,speaker,word_count_for_line
0,blossom,716
1,bubbles,626
2,buttercup,509
3,sapna,261
4,professor,244
5,princess morbucks,197
6,lester,173
7,man,120
8,woman,119
9,mojo jojo,91


In [54]:
summed_villains = lines_1998.groupby('villain')['word_count_for_line'].sum().reset_index()
summed_villains = summed_villains.sort_values('word_count_for_line', ascending=False).reset_index(drop=True)
summed_villains.head(50)

,villain,word_count_for_line
0,mojo jojo,10387
1,princess morbucks,2861
2,him,2684
3,gangreen gang,2485
4,rowdyruff boys,1593
5,smith family,970
6,amoeba boys,953
7,bernie,915
8,sedusa,817
9,dick,802


In [55]:
lines_with_seasons['villain'].value_counts().head(50)

mojo jojo            869
gangreen gang        350
rowdyruff boys       261
him                  249
princess morbucks    235
amoeba boys          157
fuzzy                111
smith family         105
sedusa                77
dick                  62
bernie                60
lenny                 25
femme fatale          23
sandman               14
gnome                  7
Name: villain, dtype: int64

In [56]:
# Good: First 9 good guys (the lower range tend either to be one-epsiode characters, or an (unrelated) aggregate)
good_aggregate_1998 = summed_speakers_1998.loc[:20, :]
good_aggregate_1998

,speaker,word_count_for_line
0,blossom,21050
1,professor,14823
2,narrator,12839
3,buttercup,12609
4,bubbles,11962
5,mojo jojo,10387
6,mayor,7573
7,ms keane,2996
8,princess morbucks,2861
9,him,2684


In [57]:
# All villains that are in top 50 most speaking people that are not an (unrelated) aggregate
villains_aggregate_1998 = summed_villains

In [58]:
good_aggregate_1998.to_json('./good_all.json', orient='records')
villains_aggregate_1998.to_json('./villains_all.json', orient='records')

In [59]:
good_aggregate_seasons_1998 = lines_1998.groupby(['speaker', 'season'])['word_count_for_line'].sum().reset_index()
good_aggregate_seasons_1998 = good_aggregate_seasons_1998.sort_values(['season', 'word_count_for_line'], ascending=[True, False]).reset_index(drop=True)
good_aggregate_seasons_1998 = good_aggregate_seasons_1998[
    (good_aggregate_seasons_1998['speaker'].isin([
        "blossom",
        "professor",
        "buttercup",
        "narrator",
        "bubbles",
        "mayor",
        "ms keane",
        "ms bellum"
    ])) | (good_aggregate_seasons_1998['word_count_for_line'] > 250)
]
good_aggregate_seasons_1998.head(50)

,speaker,season,word_count_for_line
0,blossom,1.0,3215
1,narrator,1.0,2835
2,mojo jojo,1.0,2223
3,bubbles,1.0,2162
4,buttercup,1.0,1871
5,professor,1.0,1794
6,mayor,1.0,1747
7,him,1.0,531
8,ace,1.0,406
9,ms bellum,1.0,378


In [60]:
villains_aggregate_seasons_1998 = lines_1998.groupby(['villain', 'season'])['word_count_for_line'].sum().reset_index()
villains_aggregate_seasons_1998 = villains_aggregate_seasons_1998.sort_values(['season', 'word_count_for_line'], ascending=[True, False]).reset_index(drop=True)
villains_aggregate_seasons_1998.head(50)

,villain,season,word_count_for_line
0,mojo jojo,1.0,2223
1,gangreen gang,1.0,731
2,him,1.0,531
3,fuzzy,1.0,353
4,sedusa,1.0,325
5,amoeba boys,1.0,301
6,rowdyruff boys,1.0,154
7,smith family,1.0,6
8,mojo jojo,2.0,1525
9,smith family,2.0,944


In [61]:
good_aggregate_seasons_1998.to_json('./good_seasons.json', orient='records')
villains_aggregate_seasons_1998.to_json('./villains_seasons.json', orient='records')

In [62]:
good_aggregate_episodes_1998 = lines_1998.groupby(['speaker', 'season', 'episode_nr', 'episode'])['word_count_for_line'].sum().reset_index()
good_aggregate_episodes_1998 = good_aggregate_episodes_1998.sort_values(['season', 'episode_nr', 'episode', 'word_count_for_line'], ascending=[True, True, True, False]).reset_index(drop=True)
good_aggregate_episodes_1998 = good_aggregate_episodes_1998[
    (good_aggregate_episodes_1998['speaker'].isin([
        "blossom",
        "professor",
        "buttercup",
        "narrator",
        "bubbles",
        "mayor",
        "ms keane",
        "ms bellum"
    ])) | (good_aggregate_episodes_1998['word_count_for_line'] > 100)
]
good_aggregate_episodes_1998.head(50)

,speaker,season,episode_nr,episode,word_count_for_line
0,mojo jojo,1.0,1.1,"monkey see, doggie do",325
1,narrator,1.0,1.1,"monkey see, doggie do",164
2,blossom,1.0,1.1,"monkey see, doggie do",101
3,professor,1.0,1.1,"monkey see, doggie do",47
4,mayor,1.0,1.1,"monkey see, doggie do",42
5,bubbles,1.0,1.1,"monkey see, doggie do",39
6,buttercup,1.0,1.1,"monkey see, doggie do",22
13,sedusa,1.0,1.2,mommy fearest,325
14,professor,1.0,1.2,mommy fearest,193
15,narrator,1.0,1.2,mommy fearest,160


In [63]:
villains_aggregate_episodes_1998 = lines_1998.groupby(['villain', 'season', 'episode_nr', 'episode'])['word_count_for_line'].sum().reset_index()
villains_aggregate_episodes_1998 = villains_aggregate_episodes_1998.sort_values(['season', 'episode_nr', 'episode', 'word_count_for_line'], ascending=[True, True, True, False]).reset_index(drop=True)
villains_aggregate_episodes_1998.head(50)

,villain,season,episode_nr,episode,word_count_for_line
0,mojo jojo,1.0,1.1,"monkey see, doggie do",325
1,sedusa,1.0,1.2,mommy fearest,325
2,him,1.0,3.1,octi evil,181
3,amoeba boys,1.0,3.2,geshundfight,301
4,rowdyruff boys,1.0,3.2,geshundfight,5
5,gangreen gang,1.0,4.1,buttercrush,495
6,fuzzy,1.0,4.2,fuzzy logic,214
7,gangreen gang,1.0,6.1,telephonies,236
8,mojo jojo,1.0,6.1,telephonies,102
9,him,1.0,6.1,telephonies,99


In [64]:
good_aggregate_episodes_1998.to_json('./good_episodes.json', orient='records')
villains_aggregate_episodes_1998.to_json('./villains_episodes.json', orient='records')

In [65]:
all_episodes = lines_1998.groupby(['season', 'episode_nr', 'episode', 'description'])['word_count_for_line'].sum().reset_index()
all_episodes = all_episodes.sort_values(['season', 'episode_nr', 'episode'], ascending=[True, True, True]).reset_index(drop=True)
all_episodes.head(50)

,season,episode_nr,episode,description,word_count_for_line
0,1.0,1.1,"monkey see, doggie do","In ""Monkey See, Doggie Do"", Mojo Jojo swipes s...",803
1,1.0,1.2,mommy fearest,"In ""Monkey See, Doggie Do"", Mojo Jojo swipes s...",1079
2,1.0,2.1,insect inside,"""Insect Inside"" The city of Townsville becomes...",1036
3,1.0,2.2,powerpuff bluff,"""Insect Inside"" The city of Townsville becomes...",1271
4,1.0,3.1,octi evil,"""Octi-Evil"" Him is out to break up the Powerpu...",869
5,1.0,3.2,geshundfight,"""Octi-Evil"" Him is out to break up the Powerpu...",924
6,1.0,4.1,buttercrush,"""Buttercrush"" Buttercup develops a crush on Ac...",735
7,1.0,4.2,fuzzy logic,"""Buttercrush"" Buttercup develops a crush on Ac...",601
8,1.0,5.1,boogie frights,"In ""Boogie Frights"", the Boogie Man comes up w...",1125
9,1.0,5.2,abracadaver,"In ""Boogie Frights"", the Boogie Man comes up w...",530


In [66]:
all_episodes.to_json('./all_episodes.json', orient='records')

In [67]:
top_per_good = good_aggregate_episodes_1998[good_aggregate_episodes_1998['speaker'].isin(['blossom', 'bubbles', 'buttercup', 'professor', 'narrator'])]
top_per_good = top_per_good.groupby('speaker').apply(lambda x: x.nlargest(10, 'word_count_for_line')).reset_index(drop=True)

top_per_good = top_per_good.sort_values(['speaker', 'word_count_for_line'], ascending=[True, False])

top_per_villain = villains_aggregate_episodes_1998.groupby('villain').apply(lambda x: x.nlargest(10, 'word_count_for_line')).reset_index(drop=True)
top_per_villain = top_per_villain.sort_values(['villain', 'word_count_for_line'], ascending=[True, False])
    
top_per_good

,speaker,season,episode_nr,episode,word_count_for_line
0,blossom,2.0,10.1,the powerpuff girls' best rainy day adventure ...,525
1,blossom,1.0,9.2,the bare facts,462
2,blossom,4.0,4.0,him diddle riddle,431
3,blossom,6.0,12.1,crazy mixed up puffs,396
4,blossom,4.0,12.0,power-noia,377
5,blossom,5.0,5.0,the boys are back in town,341
6,blossom,4.0,7.0,superfriends,336
7,blossom,3.0,10.1,the headsucker's moxy,310
8,blossom,5.0,13.1,curses,298
9,blossom,4.0,10.0,forced kin,284


In [68]:
top_per_villain

,villain,season,episode_nr,episode,word_count_for_line
0,amoeba boys,5.0,2.2,divide and conquer,332
1,amoeba boys,1.0,3.2,geshundfight,301
2,amoeba boys,2.0,3.1,birthday bash,170
3,amoeba boys,2.0,5.2,"you snooze, you lose",78
4,amoeba boys,6.0,10.1,sun scream,48
...,...,...,...,...,...
75,sedusa,2.0,13.1,something's a ms.,146
76,smith family,2.0,10.2,just desserts,523
77,smith family,2.0,2.2,supper villain,421
78,smith family,5.0,4.2,substitute creature,20


In [69]:
top_per_good.to_json('./good_top.json', orient='records')
top_per_villain.to_json('./villain_top.json', orient='records')

In [70]:
lines_1998[(lines_1998['season'] == 3)].groupby('episode').sum().head(50)

/var/folders/dy/hw0ckby90bxf_gm4t4jmb1yh0000gn/T/ipykernel_25533/2961070111.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  lines_1998[(lines_1998['season'] == 3)].groupby('episode').sum().head(50)


,season,episode_nr,year,word_count_for_line
episode,,,,
bought and scold,285.0,399.0,189810.0,1100
bubblevision,327.0,446.9,217782.0,701
candy is dandy,336.0,907.2,223776.0,765
catastrophe,240.0,656.0,159840.0,457
child fearing,261.0,191.4,173826.0,828
cop out,258.0,447.2,171828.0,897
criss cross crisis,843.0,843.0,561438.0,1961
equal fights,390.0,1326.0,259740.0,1255
fallen arches,369.0,135.3,245754.0,1175
